In [ ]:
import json
import shutil
import sys
from pathlib import Path

import numpy as np

In [ ]:
quality_check_metrics_file = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged_ply/metrics_new.json")
with open(quality_check_metrics_file, "r") as f:
    quality_check_metrics = json.load(f)

In [ ]:
filled_difference_fractions = []
filled_difference_fractions_full = []
for value in quality_check_metrics.values():
    # filled_difference_fractions.append(value["split"]["filled_difference_fraction"])
    filled_difference_fractions_full.append(value["full"]["filled_difference_fraction"])
# filled_difference_fractions = np.array(filled_difference_fractions)
filled_difference_fractions_full = np.array(filled_difference_fractions_full)

In [ ]:
filled_difference_fractions.mean()
np.percentile(filled_difference_fractions, [30, 50, 60])

In [ ]:
filled_difference_fractions_full.mean()
np.percentile(filled_difference_fractions_full, [30, 50, 60])

In [ ]:
# filter mlps according to filled_difference_fraction
filled_fraction_threshold = 0.1
mlps_to_keep = []
for key, value in quality_check_metrics.items():
    if value["full"]["filled_difference_fraction"] < filled_fraction_threshold:
        mlps_to_keep.append(key)
print(f"Keeping {len(mlps_to_keep)} out of {len(quality_check_metrics)} MLPs")
good_mlps_list_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged/good_shapes.lst")
with open(good_mlps_list_path, "w") as f:
    f.write("\n".join(mlps_to_keep))

In [ ]:
mlp_weights_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/mlp_weights/chair_complete_split_merged")
train_logs_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged")
for mlp_weight in train_logs_path.glob("*.pth"):
    shape_id = mlp_weight.stem.split("_")[1]
    if shape_id in mlps_to_keep:
        shutil.copy(mlp_weight, mlp_weights_path / mlp_weight.name)

In [ ]:
meshes_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/sem_seg_meshes/Chair")
meshes = []
for mesh in meshes_path.glob("*.obj"):
    shape_id = mesh.stem.split("_")[0]
    if shape_id in mlps_to_keep:
        meshes.append(mesh)
with open(meshes_path / "hyper_diff_complete_meshes.lst", "w") as f:
    f.write("\n".join([mesh.name for mesh in meshes]))

In [ ]:
mlps_trained_list_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_base_seat_sorted_classes/mlp_list.lst")
with open(mlps_trained_list_path, "r") as f:
    mlps_trained_list = f.readlines()
shapes_trained_list = []
for line in mlps_trained_list:
    shapes_trained_list.append(line.split("_")[1]) 
with open(mlps_trained_list_path.parent / "shapes_trained.lst", "w") as f:
    f.write("\n".join(shapes_trained_list))

In [ ]:
# Concatenate weight files into single folder
base_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs")
files = ["chair_complete_trial", "5", "6", "7", "8"]
merged_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged")
merged_dir.mkdir(exist_ok=True)
shape_ids = set()
for file in files:
    weight_directory = base_path / f"{file}"
    for weight_file in weight_directory.glob("*.pth"):
        shape_id = weight_file.stem.split("_")[1]
        if shape_id in shape_ids:
            continue
        shutil.copy(weight_file, merged_dir / weight_file.name)
        shape_ids.add(shape_id)

In [ ]:
shape_ids_unique = set(shape_ids)

In [ ]:
len(shape_ids)

In [ ]:
len(shape_ids_unique)

In [ ]:
import sys
sys.path.append(
    "/home/pauldelseith/HyperDiffusion_semantic_decomposition"
)
from progressbar import ProgressBar
from siren.sdf_visualization import generate_mesh_from_sdf
from hydra import compose, initialize
generated_meshes_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged_ply")
generated_meshes_shapes = [file.name.split("_")[1] for file in generated_meshes_dir.glob("*.ply")]
generated_meshes_shapes = set(generated_meshes_shapes)
config_path_relative = Path("../configs/overfitting_configs")
with initialize(version_base=None, config_path=str(config_path_relative)):
        cfg = compose(config_name="overfit_chair")
progress_bar = ProgressBar(
        maxval=len(list(merged_dir.glob("*")))
    ).start()
shapes_skipped = 0
for shape_id in shape_ids:
    if shape_id in generated_meshes_shapes:
        progress_bar.update(progress_bar.currval + 1)
        shapes_skipped += 1
        if shapes_skipped % 100 == 0:
            print(f"Skipped {shapes_skipped} shapes")
        continue
    file = list(merged_dir.glob(f"*{shape_id}*.pth"))[0]
    generate_mesh_from_sdf(file, generated_meshes_dir / file.stem, cfg, split_sdf=True)
    progress_bar.update(progress_bar.currval + 1)

In [ ]:
# get list of shapes that still have to be trained
mlp_weights_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_baseline")
shapes_trained = [file.stem.split("_")[1] for file in mlp_weights_path.glob("*.pth")]
train_mesh_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/sem_seg_meshes/")
splits = ["Chair_100000_pc_occ_in_out_True_full_split_5", "Chair_100000_pc_occ_in_out_True_full_split_6"]
shapes_to_train = []
for split in splits:
    meshes = list((train_mesh_dir / split).glob("*.npy"))
    shape_ids = [mesh.stem.split("_")[0] for mesh in meshes]
    shape_ids = set(shape_ids)
    shapes_to_train.append(list(shape_ids - set(shapes_trained)))

In [ ]:
for idx, split in enumerate(splits):
    with open(train_mesh_dir / split / "shapes_to_train.lst", "w") as f:
        for shape_id in shapes_to_train[idx]:
            f.write(str(shape_id) + '\n')


In [ ]:
generated_meshes_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged_ply")
generated_meshes_shapes = [file.name.split("_")[1] for file in generated_meshes_dir.glob("*.ply")]
generated_meshes_shapes = set(generated_meshes_shapes)

In [ ]:
len(generated_meshes_shapes)